In [62]:
import pandas as pd
import sqlglot
import numpy as np

In [63]:
df = pd.read_json('data.json')
df.head()

,question,query,generation,retrieval_method,nshot,masked,no_literals,precision,recall,f1
0,what is warfarin sodium 2.5 mg po tabs's way o...,select distinct medication.routeadmin from med...,what is the ingesting method of warfarin sodium?,column_jaccard,1,True,False,0.808309,0.662931,0.728437
1,"what is the method for ingestion of ns 1,000 ml?",select distinct medication.routeadmin from med...,what is the ingesting method of niacin?,column_jaccard,1,True,False,0.728178,0.666734,0.696103
2,how is atorvastatin calcium 80 mg po tabs taken?,select distinct medication.routeadmin from med...,what is the name of the medication containing ...,column_jaccard,1,True,False,0.716453,0.668118,0.691442
3,what is metoprolol succinate er 50 mg po tb24'...,select distinct medication.routeadmin from med...,what is the prescribing doctor's name and the ...,column_jaccard,1,True,False,0.552028,0.427494,0.481845
4,tell me the price of a procedure called agent ...,select distinct cost.cost from cost where cost...,what is the price of a procedure named agent s...,column_jaccard,1,True,False,0.872670,0.864234,0.868431


In [64]:
df = df[df['no_literals'] == False]
df = df[df['masked'] == True]
df['prec_bin'] = pd.cut(df['precision'], 10)
df['rec_bin'] = pd.cut(df['recall'], 10)

In [65]:
df.head()

,question,query,generation,retrieval_method,nshot,masked,no_literals,precision,recall,f1,prec_bin,rec_bin
0,what is warfarin sodium 2.5 mg po tabs's way o...,select distinct medication.routeadmin from med...,what is the ingesting method of warfarin sodium?,column_jaccard,1,True,False,0.808309,0.662931,0.728437,"(0.797, 0.896]","(0.598, 0.697]"
1,"what is the method for ingestion of ns 1,000 ml?",select distinct medication.routeadmin from med...,what is the ingesting method of niacin?,column_jaccard,1,True,False,0.728178,0.666734,0.696103,"(0.697, 0.797]","(0.598, 0.697]"
2,how is atorvastatin calcium 80 mg po tabs taken?,select distinct medication.routeadmin from med...,what is the name of the medication containing ...,column_jaccard,1,True,False,0.716453,0.668118,0.691442,"(0.697, 0.797]","(0.598, 0.697]"
3,what is metoprolol succinate er 50 mg po tb24'...,select distinct medication.routeadmin from med...,what is the prescribing doctor's name and the ...,column_jaccard,1,True,False,0.552028,0.427494,0.481845,"(0.498, 0.598]","(0.398, 0.498]"
4,tell me the price of a procedure called agent ...,select distinct cost.cost from cost where cost...,what is the price of a procedure named agent s...,column_jaccard,1,True,False,0.872670,0.864234,0.868431,"(0.797, 0.896]","(0.797, 0.896]"


In [66]:
df['prec-recall-diff'] = df['precision'].to_numpy() - df['recall'].to_numpy()
df = df.sort_values(by=['prec-recall-diff']).reset_index(drop=True)
df.head()

,question,query,generation,retrieval_method,nshot,masked,no_literals,precision,recall,f1,prec_bin,rec_bin,prec-recall-diff
0,how much is enoxaparin 40 mg/0.4 ml syr?,select distinct cost.cost from cost where cost...,"This query selects the unique column ""COL0"" fr...",random,0,True,False,0.445457,0.752324,0.559581,"(0.398, 0.498]","(0.697, 0.797]",-0.306866
1,what is the last ward of patient 013-36766?,select patient.wardid from patient where patie...,find the patient with the identifier 013-36766...,random,2,True,False,0.453815,0.694776,0.549020,"(0.398, 0.498]","(0.598, 0.697]",-0.240961
2,what is the price for orthopedics consultation?,select distinct cost.cost from cost where cost...,This query SELECTs all of the cost columns whe...,random,0,True,False,0.412436,0.644645,0.503036,"(0.398, 0.498]","(0.598, 0.697]",-0.232209
3,get me the top five most frequent lab tests?,"select t1.labname from ( select lab.labname, d...",tell me the five most common lab tests given t...,tfidf,2,True,False,0.618924,0.851114,0.716682,"(0.598, 0.697]","(0.797, 0.896]",-0.232190
4,how many patients were given a methemoglobin l...,select count( distinct patient.uniquepid ) fro...,count how many patients have a methemoglobin v...,random,3,True,False,0.461961,0.692513,0.554216,"(0.398, 0.498]","(0.598, 0.697]",-0.230552


In [73]:
# example with high recall but low precision
examples = list([row for _, row in df.iterrows()])

def print_row(row: pd.Series):
    for i in ['question', 'query', 'generation']:
        val = row[i]
        print(i.upper())
        if i == 'query':
            print(sqlglot.parse_one(val, read='sqlite').sql(pretty=True))
        else:
            print(val)
        print()
    
    print(
        row['retrieval_method'],
        f"{row['nshot']}-shot",
        'MASKED' if row['masked'] else 'NOT MASKED'
        )

    print(
        'P', np.round(row['precision'], 3),
        'R', np.round(row['recall'], 3),
        'F1', np.round(row['f1'], 3),
        )

In [77]:
print_row(examples[3])

QUESTION
get me the top five most frequent lab tests?

QUERY
SELECT
  t1.labname
FROM (
  SELECT
    lab.labname,
    DENSE_RANK() OVER (ORDER BY COUNT(*) DESC) AS c1
  FROM lab
  GROUP BY
    lab.labname
) AS t1
WHERE
  t1.c1 <= 5

GENERATION
tell me the five most common lab tests given to patients who have been in the hospital for at least 5 days?

tfidf 2-shot MASKED
P 0.619 R 0.851 F1 0.717


In [78]:
print_row(examples[-1])

QUESTION
how many prescriptions were issued for potassium chloride in nacl 20-0.9 meq/l-% iv soln?

QUERY
SELECT
  COUNT(*)
FROM medication
WHERE
  medication.drugname = 'potassium chloride in nacl 20-0.9 meq/l-% iv soln'

GENERATION
How many medications contains potassium chloride?

random 1-shot MASKED
P 0.732 R 0.416 F1 0.531


In [83]:
df = df.sort_values(by='f1', ascending=False).reset_index(drop=True)
df = df[df['f1'] < 0.5]
df.head()

,question,query,generation,retrieval_method,nshot,masked,no_literals,precision,recall,f1,prec_bin,rec_bin,prec-recall-diff
712,how many times has patient 010-1155 had a po l...,select count(*) from intakeoutput where intake...,find all patients who have ingested potassium ...,random,2,True,False,0.513468,0.487113,0.499944,"(0.498, 0.598]","(0.398, 0.498]",0.026355
713,when was the first time patient 021-131895 was...,select patient.hospitaldischargetime from pati...,find all patients who have a discharge date th...,column_jaccard,3,True,False,0.514143,0.486452,0.499914,"(0.498, 0.598]","(0.398, 0.498]",0.027691
714,"for patients who took anticonvulsants , what a...",select t3.treatmentname from ( select t2.treat...,find the first 5 values in the t3 table that a...,random,3,True,False,0.497744,0.502002,0.499864,"(0.398, 0.498]","(0.498, 0.598]",-0.004258
715,what is the name of the drug that was prescrib...,select t1.drugname from ( select medication.dr...,SELECT medication whose ingredients have two a...,random,1,True,False,0.553886,0.455224,0.499732,"(0.498, 0.598]","(0.398, 0.498]",0.098662
716,what was patient 027-82318's different respira...,select ( select vitalperiodic.respiration from...,find the difference in heartbeats between the ...,tfidf,3,True,False,0.582069,0.437602,0.499601,"(0.498, 0.598]","(0.398, 0.498]",0.144467


In [85]:
print_row(df.iloc[1])

QUESTION
when was the first time patient 021-131895 was discharged since 3 years ago?

QUERY
SELECT
  patient.hospitaldischargetime
FROM patient
WHERE
  patient.uniquepid = '021-131895'
  AND DATETIME(patient.hospitaldischargetime) >= DATETIME(CURRENT_TIME(), '-3 year')
ORDER BY
  patient.hospitaldischargetime
LIMIT 1

GENERATION
find all patients who have a discharge date that is within 3 years of the current time

column_jaccard 3-shot MASKED
P 0.514 R 0.486 F1 0.5
